In [1]:
import os
import pandas as pd

df_path = "data/250520_Retail (Guangdong-Guangxi)_assessment_report_Q1-2025_controlperrow_original.xlsx"
ori_df = pd.read_excel(df_path)
ori_df["company"] = "lotus_south"
ori_df["process_desc"] = ""
ori_df["rootcause_desc"] = ""

df = ori_df.copy()
print(df.shape)
df.head()

(35, 23)


,No.,User,risk_cat,risk,risk_desc,rootcause,process,likelihood_combined,impact_combined,Table Type,...,Control,Control Description,EC-Root Cause,EC-Process,Design Score,Effective Score,Control Owner,company,process_desc,rootcause_desc
0,1,caowo.user,Operational Risk,Low bargaining power with suppliers,由于门店数减少，与供应商谈判时购买总量减少，无法获得更优惠的价格或灵活的付款条件，导致成本上...,Small purchase volumes: 门店销售体量下降，缺乏对供应商的议价筹码。,Procurement: 商品部,2,2,non_financial_impact,...,执行大单品策略,改变商品经营策略，由一站式购齐改为精简商品品项，集中力量做大单品销量，获得更多的谈判能力。\n,Small purchase volumes: 门店销售体量下降，缺乏对供应商的议价筹码。,Procurement: 商品部,3,3,莲花南区生鲜商品部助理副总裁何捷伟、莲花南区非生鲜商品部助理副总裁张正夫,lotus_south,,
1,2,caowo.user,Operational Risk,Cost or expense overspending,总部和门店的费用高，直接影响盈利水平。\n,Other: 运营效率低下或预算管理不善。,Marketing and Sales: 营运部,2,1,financial_impact,...,成本控制与预算管理,提升运营效率，降低费用；定期审查预算执行情况。\n,Other: 运营效率低下或预算管理不善。,Marketing and Sales: 营运部,3,3,莲花南区 B2C支 持中心助理副总裁肖训文,lotus_south,,
2,3,caowo.user,Operational Risk,New business execution failure,超市尝试推出新的业务模式，但由于执行、能力、技术和市场推广不足等原因，未能达到预期效果，从而...,Lack of market understanding: Inability to att...,Marketing and Sales: 市场部,2,2,financial_impact,...,新业务试点与推广计划,加强市场调研和技术测试；强化业务能力培训和追踪执行；制定详细的推广计划并分阶段实施。\n,Lack of market understanding: Inability to att...,Marketing and Sales: 市场部,3,3,莲花南区市场部和 O2O部助理副总裁李铮铮,lotus_south,,
3,4,caowo.user,Operational Risk,Poor product quality,销售商品被发现存在质量问题，导致顾客投诉并要求退货，超市声誉受损。\n,Inadequate or ineffective quality control proc...,Quality Assurance: QAQC,2,1,financial_impact,...,QAQC\t\t供应商质量管理与检验流程优化,加强供应商质量管理；完善进货检验和抽检流程。\n,Inadequate or ineffective quality control proc...,Quality Assurance: QAQC,3,3,QAQC 负责人吕永权,lotus_south,,
4,5,caowo.user,Operational Risk,Understocking inventory,因销量上涨订货不及时、供应商无法及时供货等原因，导致货架上商品短缺，顾客无法购买到所需商品。\n,"Inaccurate demand forecasting: 供应链中断、需求预测不准确, ...",Procurement: 商品部,4,3,financial_impact,...,库存优化与供应链备份,优化自动补货模型；建立多供应商备份机制。\n,"Inaccurate demand forecasting: 供应链中断、需求预测不准确, ...",Procurement: 商品部,3,3,莲花南区生鲜商品部助理副总裁何捷伟、莲花南区非生鲜商品部助理副总裁张正夫,lotus_south,,


In [2]:
file_name = os.path.basename(df_path)
date_stamp = file_name.split("_")[0]
date_stamp

'250520'

In [3]:
# calc risk score
# Select columns that start with 'financial' but do not end with '_note'
# riskscore_columns = df.loc[:, (df.columns.str.startswith('Financial') | df.columns.str.startswith('NonFinancial') | df.columns.str.startswith('Likelihood')) & ~df.columns.str.endswith('_notes') & ~df.columns.str.endswith('_justification')]
df = ori_df.copy()


df["risk_score"] = df["likelihood_combined"] * df["impact_combined"]


# Define the function to determine the risk level
def determine_risk_level(row, type):
    if type == "fin":
        riskscore = "risk_score_fin"
        riskimpact = "impact_fin_combined"
    elif type == "nonfin":
        riskscore = "risk_score_nonfin"
        riskimpact = "impact_nonfin_combined"
    else:
        riskscore = "risk_score"
        riskimpact = "impact_combined"
    if row[riskscore] >= 20:
        return 4
    elif 10 <= row[riskscore] <= 16:
        return 3
    elif 4 <= row[riskscore] <= 9:
        return 2
    elif (0 < row[riskscore] < 4) or (row[riskscore] == 4 and row[riskimpact] == 2):
        return 1
    elif row[riskscore] == 0:
        return 0
    # else:
    #     return 'Undefined'


# Apply the function to each row in the DataFrame
df["risk_level"] = df.apply(determine_risk_level, type="combined", axis=1)

In [4]:
df["risk_level"].value_counts()

risk_level
2    19
1    13
3     3
Name: count, dtype: int64

In [5]:
debug_df = df[df["company"] == "lotus_south"]
debug_df = debug_df[debug_df["risk"].str.lower() == "poor supply quality"]
debug_df[["risk", "risk_score", "likelihood_combined", "impact_combined"]]

,risk,risk_score,likelihood_combined,impact_combined


In [6]:
df2 = df.groupby(["company", "risk"], as_index=False).agg(
    {
        "risk_desc": set,
        "risk_cat": "first",
        # "iscustom": set,  #'first', # risk
        # "company_sector": set,
        # "risk_subcat": set,
        # "control_name": set,
        # "control_des": set,
        "rootcause": set,
        "rootcause_desc": set,
        "process": set,
        "process_desc": set,
        "risk_level": max,
        "likelihood_combined": max,
        "impact_combined": max,
        # "impact_fin_combined": max,
        # "impact_nonfin_combined": max,
    }
)
# TODO: check if other df# are recalculating this since max(impact)*max(likelihood) != max(risk_score)
df2["risk_score"] = df2["impact_combined"] * df2["likelihood_combined"]
# df2["risk_score_fin"] = df2["impact_fin_combined"] * df2["likelihood_combined"]
# df2["risk_score_nonfin"] = df2["impact_nonfin_combined"] * df2["likelihood_combined"]

df2["risk_level"] = df2.apply(determine_risk_level, type="combine", axis=1)
# df2["risk_level_fin"] = df2.apply(determine_risk_level, type="fin", axis=1)
# df2["risk_level_nonfin"] = df2.apply(determine_risk_level, type="nonfin", axis=1)

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50631/2672502393.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df2 = df.groupby(["company", "risk"], as_index=False).agg(


In [7]:
print(df2.shape)
df2.head()

(35, 12)


,company,risk,risk_desc,risk_cat,rootcause,rootcause_desc,process,process_desc,risk_level,likelihood_combined,impact_combined,risk_score
0,lotus_south,Business interruption from fire hazards,{火灾事故导致的业务中断，可能会对建筑物、财产和各种设备造成损害。这种损害可能需要暂时或永久...,Operational Risk,{Insufficient Maintenance of Fire Safety Equip...,{},"{Facilities Management: 安全维保部, Maintenance: 安全...",{},2,2,2,4
1,lotus_south,Business interruption from natural disasters,{台风/暴雨自然灾害风险造成中断营业\n},Operational Risk,{Lack of Emergency Response Training: 不可抗力因素，缺...,{},"{Health, Safety, and Environment (HSE): 安全维保部}",{},1,2,1,2
2,lotus_south,Business interruption from pandemic or epidemic,{疫情或流行病导致的商业中断涉及传染病的爆发，这可能导致关闭、隔离或减少劳动力可用性，从而影...,Operational Risk,{Other: 对健康危机准备不足；依赖实体店运营。},{},"{Health, Safety, and Environment (HSE): 安全维保部}",{},2,2,2,4
3,lotus_south,Carbon emission control,{政府控制碳排放的政策、法规或要求而产生的运营成本上涨；或商品采购成本上涨。\n},Strategic Risk,{Other: 缺乏可持续实践；依赖非环保运营。},{},{Marketing and Sales: 营运部},{},1,1,1,1
4,lotus_south,Cost or expense overspending,{总部和门店的费用高，直接影响盈利水平。\n},Operational Risk,{Other: 运营效率低下或预算管理不善。},{},{Marketing and Sales: 营运部},{},1,2,1,2


In [8]:
df2.company.value_counts()

company
lotus_south    35
Name: count, dtype: int64

In [9]:
df3 = df2[
    df2["company"].isin(
        [
            "lotus_south",
        ]
    )
]
print(df3.shape)
df3.company.value_counts()

(35, 12)


company
lotus_south    35
Name: count, dtype: int64

In [10]:
df3.head()

,company,risk,risk_desc,risk_cat,rootcause,rootcause_desc,process,process_desc,risk_level,likelihood_combined,impact_combined,risk_score
0,lotus_south,Business interruption from fire hazards,{火灾事故导致的业务中断，可能会对建筑物、财产和各种设备造成损害。这种损害可能需要暂时或永久...,Operational Risk,{Insufficient Maintenance of Fire Safety Equip...,{},"{Facilities Management: 安全维保部, Maintenance: 安全...",{},2,2,2,4
1,lotus_south,Business interruption from natural disasters,{台风/暴雨自然灾害风险造成中断营业\n},Operational Risk,{Lack of Emergency Response Training: 不可抗力因素，缺...,{},"{Health, Safety, and Environment (HSE): 安全维保部}",{},1,2,1,2
2,lotus_south,Business interruption from pandemic or epidemic,{疫情或流行病导致的商业中断涉及传染病的爆发，这可能导致关闭、隔离或减少劳动力可用性，从而影...,Operational Risk,{Other: 对健康危机准备不足；依赖实体店运营。},{},"{Health, Safety, and Environment (HSE): 安全维保部}",{},2,2,2,4
3,lotus_south,Carbon emission control,{政府控制碳排放的政策、法规或要求而产生的运营成本上涨；或商品采购成本上涨。\n},Strategic Risk,{Other: 缺乏可持续实践；依赖非环保运营。},{},{Marketing and Sales: 营运部},{},1,1,1,1
4,lotus_south,Cost or expense overspending,{总部和门店的费用高，直接影响盈利水平。\n},Operational Risk,{Other: 运营效率低下或预算管理不善。},{},{Marketing and Sales: 营运部},{},1,2,1,2


In [11]:
df3.columns

Index(['company', 'risk', 'risk_desc', 'risk_cat', 'rootcause',
       'rootcause_desc', 'process', 'process_desc', 'risk_level',
       'likelihood_combined', 'impact_combined', 'risk_score'],
      dtype='object')

In [12]:
select_columns = [
    "company",
    "risk_cat",
    "risk",
    "risk_desc",
    "rootcause",
    "rootcause_desc",
    "process",
    "process_desc",
    "risk_level",
]
df4 = df3[select_columns]
print(df4.shape)
df4.head()

(35, 9)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,lotus_south,Operational Risk,Business interruption from fire hazards,{火灾事故导致的业务中断，可能会对建筑物、财产和各种设备造成损害。这种损害可能需要暂时或永久...,{Insufficient Maintenance of Fire Safety Equip...,{},"{Facilities Management: 安全维保部, Maintenance: 安全...",{},2
1,lotus_south,Operational Risk,Business interruption from natural disasters,{台风/暴雨自然灾害风险造成中断营业\n},{Lack of Emergency Response Training: 不可抗力因素，缺...,{},"{Health, Safety, and Environment (HSE): 安全维保部}",{},1
2,lotus_south,Operational Risk,Business interruption from pandemic or epidemic,{疫情或流行病导致的商业中断涉及传染病的爆发，这可能导致关闭、隔离或减少劳动力可用性，从而影...,{Other: 对健康危机准备不足；依赖实体店运营。},{},"{Health, Safety, and Environment (HSE): 安全维保部}",{},2
3,lotus_south,Strategic Risk,Carbon emission control,{政府控制碳排放的政策、法规或要求而产生的运营成本上涨；或商品采购成本上涨。\n},{Other: 缺乏可持续实践；依赖非环保运营。},{},{Marketing and Sales: 营运部},{},1
4,lotus_south,Operational Risk,Cost or expense overspending,{总部和门店的费用高，直接影响盈利水平。\n},{Other: 运营效率低下或预算管理不善。},{},{Marketing and Sales: 营运部},{},1


In [13]:
# select first row of column risk_desc
cell = df4.iloc[4, 4]
# cell = df4.iloc[2, 3]
# cell = df4.iloc[3, 3]
print(type(cell))
print(cell)
import json
import traceback
from typing import Union


def list_string_to_list(list_string) -> tuple[bool, Union[list, str]]:
    is_list = False
    if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
        (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
    ):

        return is_list, list_string
    # print(f"pass list_string :{type(list_string)} {list_string}")
    is_list = True
    # swap "'" to '"' and swap '"' to "'"
    list_string = list_string.replace('"', "###")
    list_string = list_string.replace("'", '"')
    list_string = list_string.replace("###", "'")
    try:
        list_cell = json.loads(list_string)
    except:
        is_list = False
        # print(f"here list_string : {list_string}")
        # print(traceback.format_exc())
        return is_list, list_string
    return is_list, list_cell


def cell_to_list(cell):
    empty_element_list = ["", "-"]
    if type(cell) == set:
        new_cell = []
        for list_string_candidate in cell:
            if list_string_candidate in empty_element_list:
                continue
            try:
                # if list_string count "'" or '"' have to be even and not 0
                # print(f"list_string_candidate : >>>{list_string_candidate}<<<")

                is_list, new_list_string_candidate = list_string_to_list(
                    list_string_candidate
                )
                # print(new_list_string_candidate == list_string_candidate)
                # print("is_list", is_list)
                # print(f"new_list_string_candidate : >>>{new_list_string_candidate}<<<")
                if is_list is False:
                    new_cell.append(new_list_string_candidate)
                    continue
                for i in new_list_string_candidate:
                    if i in empty_element_list:
                        continue

                    is_list, i = list_string_to_list(i)
                    # print(f"i : {is_list}>>>{i}<<<")
                    if is_list is False:
                        new_cell.append(i)
                        continue
                    new_cell.extend(i)

            except:
                # print(f"list_string : {list_string}")
                # print(traceback.format_exc())
                new_cell.append(list_string)
        return new_cell
    return cell


result = cell_to_list(cell)
print(cell_to_list(cell))
print(type(cell_to_list(cell)))

# print(cell_to_list(cell)[0])
# print(type(cell_to_list(cell)[0]))

<class 'set'>
{'Other: 运营效率低下或预算管理不善。'}
['Other: 运营效率低下或预算管理不善。']
<class 'list'>


In [14]:
result

['Other: 运营效率低下或预算管理不善。']

In [15]:
list_string = "ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้นตอน ซึ่งขึ้นอยู่แต่ละปัญหาของสินค้า"
if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
    (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
):
    print("yes")

In [16]:
# df5 is df4 with pass each cell with cell_to_list
# convert set of list_string to list
df5 = df4.applymap(cell_to_list)
df5.head()

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50631/139688506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df5 = df4.applymap(cell_to_list)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,lotus_south,Operational Risk,Business interruption from fire hazards,[火灾事故导致的业务中断，可能会对建筑物、财产和各种设备造成损害。这种损害可能需要暂时或永久...,[Insufficient Maintenance of Fire Safety Equip...,[],"[Facilities Management: 安全维保部, Maintenance: 安全...",[],2
1,lotus_south,Operational Risk,Business interruption from natural disasters,[台风/暴雨自然灾害风险造成中断营业\n],[Lack of Emergency Response Training: 不可抗力因素，缺...,[],"[Health, Safety, and Environment (HSE): 安全维保部]",[],1
2,lotus_south,Operational Risk,Business interruption from pandemic or epidemic,[疫情或流行病导致的商业中断涉及传染病的爆发，这可能导致关闭、隔离或减少劳动力可用性，从而影...,[Other: 对健康危机准备不足；依赖实体店运营。],[],"[Health, Safety, and Environment (HSE): 安全维保部]",[],2
3,lotus_south,Strategic Risk,Carbon emission control,[政府控制碳排放的政策、法规或要求而产生的运营成本上涨；或商品采购成本上涨。\n],[Other: 缺乏可持续实践；依赖非环保运营。],[],[Marketing and Sales: 营运部],[],1
4,lotus_south,Operational Risk,Cost or expense overspending,[总部和门店的费用高，直接影响盈利水平。\n],[Other: 运营效率低下或预算管理不善。],[],[Marketing and Sales: 营运部],[],1


In [17]:
# df6 convert list to string with join
df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)
df6.head()

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50631/3479902738.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,lotus_south,Operational Risk,Business interruption from fire hazards,火灾事故导致的业务中断，可能会对建筑物、财产和各种设备造成损害。这种损害可能需要暂时或永久停...,Insufficient Maintenance of Fire Safety Equipm...,,"Facilities Management: 安全维保部, Maintenance: 安全维保部",,2
1,lotus_south,Operational Risk,Business interruption from natural disasters,台风/暴雨自然灾害风险造成中断营业\n,Lack of Emergency Response Training: 不可抗力因素，缺乏...,,"Health, Safety, and Environment (HSE): 安全维保部",,1
2,lotus_south,Operational Risk,Business interruption from pandemic or epidemic,疫情或流行病导致的商业中断涉及传染病的爆发，这可能导致关闭、隔离或减少劳动力可用性，从而影响...,Other: 对健康危机准备不足；依赖实体店运营。,,"Health, Safety, and Environment (HSE): 安全维保部",,2
3,lotus_south,Strategic Risk,Carbon emission control,政府控制碳排放的政策、法规或要求而产生的运营成本上涨；或商品采购成本上涨。\n,Other: 缺乏可持续实践；依赖非环保运营。,,Marketing and Sales: 营运部,,1
4,lotus_south,Operational Risk,Cost or expense overspending,总部和门店的费用高，直接影响盈利水平。\n,Other: 运营效率低下或预算管理不善。,,Marketing and Sales: 营运部,,1


In [18]:
df7 = df6.copy()


# new_rootcause is 'rootcause : ' + rootcause column +\n+ 'rootcause_desc : ' rootcause_desc if it not '' empty string
# fundction to apply
def merge_rootcause(row):
    rootcause = row["rootcause"]
    rootcause_desc = row["rootcause_desc"]
    if rootcause == "" and rootcause_desc == "":
        return ""
    elif rootcause == "":
        return "rootcause_desc :" + rootcause_desc
    elif rootcause_desc == "":
        return "rootcause :" + rootcause
    else:
        return "rootcause :" + rootcause + "\n" + "rootcause_desc :" + rootcause_desc


df7["new_rootcause"] = df7.apply(merge_rootcause, axis=1)


def merge_process(row):
    process = row["process"]
    process_desc = row["process_desc"]
    if process == "" and process_desc == "":
        return ""
    elif process == "":
        return "process_desc :" + process_desc
    elif process_desc == "":
        return "process :" + process
    else:
        return "process :" + process + "\n" + "process_desc :" + process_desc


df7["new_process"] = df7.apply(merge_process, axis=1)
# remove rootcause and process column
df7.drop(
    ["rootcause", "rootcause_desc", "process", "process_desc"], axis=1, inplace=True
)
# rename new_rootcause and new_process to rootcause and process
df7.rename(
    columns={"new_rootcause": "rootcause", "new_process": "process"}, inplace=True
)
df7.head()

,company,risk_cat,risk,risk_desc,risk_level,rootcause,process
0,lotus_south,Operational Risk,Business interruption from fire hazards,火灾事故导致的业务中断，可能会对建筑物、财产和各种设备造成损害。这种损害可能需要暂时或永久停...,2,rootcause :Insufficient Maintenance of Fire Sa...,"process :Facilities Management: 安全维保部, Mainten..."
1,lotus_south,Operational Risk,Business interruption from natural disasters,台风/暴雨自然灾害风险造成中断营业\n,1,rootcause :Lack of Emergency Response Training...,"process :Health, Safety, and Environment (HSE)..."
2,lotus_south,Operational Risk,Business interruption from pandemic or epidemic,疫情或流行病导致的商业中断涉及传染病的爆发，这可能导致关闭、隔离或减少劳动力可用性，从而影响...,2,rootcause :Other: 对健康危机准备不足；依赖实体店运营。,"process :Health, Safety, and Environment (HSE)..."
3,lotus_south,Strategic Risk,Carbon emission control,政府控制碳排放的政策、法规或要求而产生的运营成本上涨；或商品采购成本上涨。\n,1,rootcause :Other: 缺乏可持续实践；依赖非环保运营。,process :Marketing and Sales: 营运部
4,lotus_south,Operational Risk,Cost or expense overspending,总部和门店的费用高，直接影响盈利水平。\n,1,rootcause :Other: 运营效率低下或预算管理不善。,process :Marketing and Sales: 营运部


In [19]:
risk_df_path = "data/RMI-V2-Translate_20250508.xlsx"
risk_df = pd.read_excel(risk_df_path, sheet_name="Risks")
risk_cause_mapping_df = pd.read_excel(risk_df_path, sheet_name="Risk_Cause_mapping")

selected_cols = [
    "Risk-EN",
    "Description-EN",
]

# risk_df["Risk-category"]# replace Operational risk with Operational Risk
risk_df["Risk-category"] = risk_df["Risk-category"].replace(
    "Operational risk", "Operational Risk"
)
for idx, row in risk_df.iterrows():
    risk_en = row["Risk-EN"]
    # risk_cause_mapping_df column RiskName get all RiskCause
    risk_cause_mapping_df_row = risk_cause_mapping_df[
        risk_cause_mapping_df["RiskName"] == risk_en
    ]
    # get all RiskCause
    risk_cause_list = risk_cause_mapping_df_row["RiskCause"].tolist()
    risk_cause_str = "\n".join(["- " + i for i in risk_cause_list])
    # add risk_cause_str to column Root cause-EN
    risk_df.loc[idx, "Root cause-EN"] = risk_cause_str


print(risk_df.columns)
risk_and_risk_desc_df = risk_df[selected_cols]
# loop each row in risk_and_risk_desc_df:
# apply strip to all columns
risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())

risk_and_risk_desc_df.head()
# create map_dict
risk_to_desc_dict = risk_and_risk_desc_df.set_index("Risk-EN")[
    "Description-EN"
].to_dict()
len(risk_to_desc_dict)

/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Index(['GroupingIndex', 'Grouping Question Name', 'Code', 'Risk-category',
       'Risk-Sub-Cat-EN', 'Risk-EN', 'Risk-TH', 'Risk-CN', 'Description-EN',
       'Description-TH', 'Description-CN', 'Unnamed: 11', 'Root cause-EN'],
      dtype='object')


/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50631/3047557379.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())


314

In [20]:
df8 = df7.copy()
# print number cell == NaN
print(df8.isnull().sum().sum())
# risk column map with risk_to_desc_dict
# apply strip to df8["risk"]
df8["risk"] = df8["risk"].str.strip()
df8["risk_desc_catalog"] = df8["risk"].map(risk_to_desc_dict)

df8.head()
# is nan in risk_desc_catalog
print(df8["risk_desc_catalog"].isnull().sum())
# fill risk_desc_catalog nan with ""
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].fillna("")
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].str.strip()
# print row that risk_desc_catalog is nan
df8[df8["risk_desc_catalog"].isnull()]
# concat risk_desc_catalog and risk_desc
df8["risk_desc"] = df8["risk_desc_catalog"] + " " + df8["risk_desc"]
# strip df8["risk_desc"]
df8["risk_desc"] = df8["risk_desc"].str.strip()
# remove risk_desc_catalog
df8.drop("risk_desc_catalog", axis=1, inplace=True)
all_risk_cat = risk_df["Risk-category"].unique().tolist()
# check is all df8['risk_cat'] in all_risk_cat
assert all([x in all_risk_cat for x in df8["risk_cat"].unique().tolist()])
# add new data to df8 from risk_and_risk_cat_and_risk_desc_df
print(df8.shape)

new_rows = []
for index, row in risk_df.iterrows():
    risk_cat = row["Risk-category"]
    risk_desc = row["Description-EN"]
    risk_name = row["Risk-EN"]
    risk_rootcause = row["Root cause-EN"]
    data = {
        # company	risk_cat	risk	risk_desc	rootcause	process
        "company": f"risk_catalog-{date_stamp}",
        "risk_cat": risk_cat,
        "risk": risk_name,
        "risk_desc": risk_desc,
        "rootcause": risk_rootcause,
        "process": "",
        "risk_level": 0,
    }
    new_rows.append(data)
df8 = pd.concat([df8, pd.DataFrame(new_rows)], ignore_index=True)
print(df8.shape)
print("nan", df8.isnull().sum().sum())

df8.head()

0
0
(35, 7)
(349, 7)
nan 0


,company,risk_cat,risk,risk_desc,risk_level,rootcause,process
0,lotus_south,Operational Risk,Business interruption from fire hazards,The business interruption due to a fire incide...,2,rootcause :Insufficient Maintenance of Fire Sa...,"process :Facilities Management: 安全维保部, Mainten..."
1,lotus_south,Operational Risk,Business interruption from natural disasters,Business interruption from natural disasters s...,1,rootcause :Lack of Emergency Response Training...,"process :Health, Safety, and Environment (HSE)..."
2,lotus_south,Operational Risk,Business interruption from pandemic or epidemic,Business interruption from pandemic or epidemi...,2,rootcause :Other: 对健康危机准备不足；依赖实体店运营。,"process :Health, Safety, and Environment (HSE)..."
3,lotus_south,Strategic Risk,Carbon emission control,"Risk arising from policies, regulations, or re...",1,rootcause :Other: 缺乏可持续实践；依赖非环保运营。,process :Marketing and Sales: 营运部
4,lotus_south,Operational Risk,Cost or expense overspending,The risk that the organization incurs cost or ...,1,rootcause :Other: 运营效率低下或预算管理不善。,process :Marketing and Sales: 营运部


In [21]:
all([x in all_risk_cat for x in df8["risk_cat"].unique().tolist()])

True

In [22]:
all_risk_cat

['Operational Risk',
 'Strategic Risk',
 'Market Risk',
 'Credit Risk',
 'Liquidity Risk',
 'A. Setting up business\n\n',
 'B. Business operation',
 'C. Compliance and regulator',
 'D. Policy',
 'E. Raising Concerns and Seeking Guidance',
 'F. Review, Assessment and Improvement']

In [23]:
df8["risk_cat"].unique().tolist()

['Operational Risk',
 'Strategic Risk',
 'Credit Risk',
 'Market Risk',
 'Liquidity Risk',
 'A. Setting up business\n\n',
 'B. Business operation',
 'C. Compliance and regulator',
 'D. Policy',
 'E. Raising Concerns and Seeking Guidance',
 'F. Review, Assessment and Improvement']

In [24]:
df.shape

(35, 25)

In [25]:
df8.shape

(349, 7)

In [26]:
df_final = df8.copy()

In [27]:
# convert df6:dataframe to data_dict
result_list = df_final.to_dict(orient="records")
result_list[:3]

[{'company': 'lotus_south',
  'risk_cat': 'Operational Risk',
  'risk': 'Business interruption from fire hazards',
  'risk_desc': 'The business interruption due to a fire incident that occurs within the workplace premises can result in damage to buildings, property, and various equipment. This damage may necessitate a temporary or permanent cessation of business operations.\n\nBusiness interruption: refers to a period during which normal business operations are disrupted or halted, resulting in reduced operational efficiency, loss of revenue, or increased costs. 火灾事故导致的业务中断，可能会对建筑物、财产和各种设备造成损害。这种损害可能需要暂时或永久停止商业运营。',
  'risk_level': 2,
  'rootcause': 'rootcause :Insufficient Maintenance of Fire Safety Equipment: 消防措施不足；电气系统维护不当。',
  'process': 'process :Facilities Management: 安全维保部, Maintenance: 安全维保部'},
 {'company': 'lotus_south',
  'risk_cat': 'Operational Risk',
  'risk': 'Business interruption from natural disasters',
  'risk_desc': 'Business interruption from natural disasters such

In [28]:
# save it as json
import os

dir_path = os.getcwd()
save_file_path = f"{dir_path}/result/250520-company_risk_data.json"
# save result_list as json
import json

with open(save_file_path, "w") as f:
    json.dump(result_list, f, ensure_ascii=False)